<a href="https://colab.research.google.com/github/SamuelVanWilson/Machine-Learning-Daily-Projects/blob/main/DAY_6_TRANSFER_LEARNING_KLASIFIKASI_GAMBAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MENGIDENTIFIKASI GAMBAR DENGAN TEKNIK TRANFSER LEARNING YANG MENGGUNAKAN DATA CIFAR-10**

###IMPORT LIBRARY

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt

library `tensorflow` digunakan untuk membangun model dalam pembuatan machine learning

`from tensorflow.keras.layers import Dense, GlobalAveragePooling2D`
- `Dense` digunakan untuk menghubungkan layer ke layer selanjutnya, ini berguna untuk output layer klasifikasi akhir
- `GlobalAveragePooling2D` kegunaannya mengubah output 3D dari layer convutional menjadi vektor 1D (ini menggantikan fungsi `Flatten` yang mengurangi dimensi tapi kalo gunain `Flatten` akan menambah parameter, kita tidak menginginkan itu)

`from tensorflow.keras.models import Model` kegunaannya membangun model menggunakan **API fungsional** yang memungkinkan menghubungkan layer dari model pra-latih ke layer baru yang kita tambahkan

`from tensorflow.keras.applications import MobileNetV2` ini mengimpor arsitektur model `MobileNetV2` yang sudah dilatih sebelumnya(*pre-trained*) menggunakan dataset `imageNet`

`from tensorflow.keras.datasets import cifar10` untuk mengambil dataset `cifar10` dari server

`from tensorflow.keras.utils import to_categorical` berguna untuk melakukan teknik *one-hot encoding*

---

library `numpy` digunakan untuk komputasi numerik, karena library ini lebih stabil untuk aritmatika pada pengembangan model dibandingan aritmatika python biasa

---

library `matplotlib` berguna untuk visualisasi grafik data

###MENGAMBIL DAN PREPROCESS DATA

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = tf.image.resize(x_train, (255, 255))
x_test = tf.image.resize(x_test, (255, 255))

x_train = x_train / 127.5 - 1.0
x_test = x_test / 127.5 - 1.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 141s 1us/step


pertama-tama data dipecah dulu sebagai data training yakni variabel `x_train` untuk fitur/input dan `y_train` untuk label/output, terus dibagi lagi untuk data test yakni variabel `x_test` untuk fitur/input dan `y_test` untuk label/output. ini bisa dibagi karena kita membagi nilai menggunakan fungsi `cifar10.load_data()`

---

`tf.image.resize()` resize gambar dari ukuran 32x32 menjadi ukuran 224x224 karena model `MobileNetV2` mengharapkan input ukuran 224x224w

---

`x_train / 127.5 - 1.0` menormalisasi nilai pix rentang dari 0-255 menjadi -1 sampai 1 pada data training

---

`to_categorical(y_train, 10)` melakukan one-hot encoding pada menjadi 10 kelas untuk output/label pada data test

###MENGUNAKAN MODEL PRA-LATIH

In [ ]:

base_model = MobileNetV2(
 weights='imagenet',
 input_shape=(224, 224, 3),
 include_top=False
)

base_model.training = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='rule')(x)
prediction = Dense(10, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=prediction)

kita menggunakan model `MobileNetV2` karena model ini sudah dilatih sebelumnya untuk mengenali ribuan objek gambar(seperti kucing, mobil, dll) dari dataset **ImageNet**

kita pakai *otaknya* `MobileNetV2`, tapi **buang layer terakhirnya** (karena tugas kita beda: klasifikasi 10 kelas CIFAR-10, bukan 1000 kelas ImageNet)

---

variabel `base_model` digunakan untuk menyimpan model yang sudah dibuat. model `MobileNetV2` menyimpan parameter berupa:
- `weights='imagenet'` menggunakan bobot dari ImageNet
- `input_shape=(224, 224, 3)` model ini menerima gambar ukuran 224x224 piksel dengan 3 channel (RGB)
- `include_top=False` menghapus layer klasifikasi asli `MobileNetV2` dan kita ganti dengan layer kita

---

`base_model.training = False` Membekukan semua layer di MobileNetV2(kita ingin mempertahankan "pengetahuan" yang sudah dipelajari model ini) agar tidak diubah selama pelatihan

---

`GlobalAveragePooling2D()(base_model.output)` mengubah output dari model `MobileNetV2` menjadi output vektor yang sederhana

---

`Dense(256, activation='rule')(x)` fungsi *Dense* dalam variabel `x` berguna untuk model mempelajari 256 neuron untuk mempelajari pola baru dari data CIFAR-10(karena parameter 256), selanjutnya parameter ke 2, `activation='rule'` untuk aktivasi non-linearitas

---

sedangkan `Dense(10, activation="softmax")(x)` berguna untuk layer klasifikasi akhir dengan 10 neuoran(sesuai 10 kelas pada dataset **CIFAR-10**), untuk parameter yang ke-2 `activation="softmax"` berguna untuk menghasilkan probabilitas untuk setiap kelas

---

kenapa dalam fungsi `Dense()` diakhirnya ada variabel `x` seperti ini `Dense()(x)`? itu karena kita menggunakan sintaksis **keras** untuk menghubungkan ke layer-layer tersebut kedalam input
- `Dense()`: mendefinisikan layer
- `(x)`: menghubungkan layer tersebut ke input(output dari layer sebelumnya (x) dimasukkan ke layer Dense ini, lalu hasilnya disimpan kembali ke variabel `x` untuk layer berikutnya)

---

`Model(inputs=base_model.input, outputs=predictions)` digunakan untuk menghubungkan input `MobileNetV2` dengan layer yang baru ditambahkan, sehingga menjadi model yang kita inginkan

###MELATIH & MENGOMPILASI MODEL

In [ ]:

model.compile(
 optimizer='adam',
 loss='categorical_crossentropy',
 metrics=['accuracy']
)

history = model.fit(
 x_train, y_train,
 epochs=10,
 batch_size=32,
 validation_split=0.2
)

####**Mengompilasi Model**

- **optimizer**: algoritma untuk memperbarui bobot model
- **loss function**: fungsi untuk mengukur kesalahan prediksi model
- **metrics**: metrik untuk mengevaluasi performa model

**parameter compile**:
1. `optimizer='adam'` digunakan untuk menyesuaikan bobot model untuk meminimalkan kesalahan. `adam` adalah optimizer yang populer karena cepat dan efisien.
2. `loss='categorical_crossentropy'` digunakan untuk mengukur kesalahan prediksi untuk masalah klasifikasi multi-kelas
3. `metrics=['accuracy']` digunakan untuk mengukur akurasi
---

####**Melatih Model**

variabel `history` untuk menyimpan riwayat pelatihan model, parameternya:

- `x_train` fitur yang digunakan untuk dilatih model
-`y_train` label yang digunakan untuk dilatih model
- `epochs` digunakan untuk mengulang model selama berlatih, karena epochsnya 5 kali maka, model berlatih dengan data training secara 5 kali berulang
- `batch_size=32` artinya model akan **memproses 32 gambar sekaligus** sebelum memperbarui bobotnya(bayangkan belajar 32 soal dulu → cek jawaban → perbaiki kesalahan → ulangi. lebih efisien daripada belajar 50.000 soal sekaligus)
-`validation_split=0.2` digunakan untuk validasi model selama latihan, ini seperti "ujian kecil" selama latihan model berjalan, karena nilainya `0.2` maka model akan melakukan ujian kecil menggunakan 20% data training

###EVALUASI MODEL

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Akurasi Test: {test_acc:.4f}")

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'model' is not defined

`model.evaluate()` digunakan untuk mengevaluasi model, ini seperti *tes ujian akhir* bagi model untuk melihat seberapa baik model bekerja, karena *method* ini mengembalikan berupa 2 nilai yaitu **nilai akurasi** yang disimpan kedalam variabel `test_acc` dan **nilai kesalahan selama tes** yang disimpan kedalam variabel `test_loss`, jadi tahap ini merupakan tahap model memakai data test, karena parameter permata menggunakan **fitur data test** yakni `x_test` dan parameter kedua **label data test** yakni `y_test`

###TES PREDIKSI

In [ ]:

indices = np.random.choice(len(X_test), 5)

for i in indices:
    img = X_test[i]
    true_label = np.argmax(y_test[i])

    pred = model.predict(img[np.newaxis, ...])
    pred_label = np.argmax(pred)

    plt.imshow((img + 1) / 2)
    plt.title(f"True: {true_label}, Pred: {pred_label}")
    plt.show()

variabel `indices` dugunakan untuk memilih 5 angka random, yang akan digunakan untuk memilih data secara acak sesuai nilai maksimum dataset, karena menggunakan fungsi `np.random.choice()`

---

perulangan `for i in indices:` digunakan untuk menampilkan gambar tulis tangan dan hasil prediksi model *satu persatu*